# 介绍

>ChatGPT
>要判断一句话是属于真实世界还是虚拟仙侠世界，可以考虑以下几个方面：
>
>上下文：观察这句话所处的上下文环境。如果这句话出现在现实生活中的场景或讨论中，那么它很可能属于真实世界。如果这句话出现在描述虚构故事、游戏或小说的背景下，那么它可能属于虚拟仙侠世界。
>
>内容与现实的符合度：思考这句话所陈述的内容是否与我们在真实世界中的经验和认知相符。如果这句话提到了超自然的力量、仙侠世界中特有的事物、人物或事件，那么它可能属于虚拟仙侠世界。如果这句话描述了我们在现实世界中常见的事物、人物或事件，那么它更有可能属于真实世界。
>
>可验证性：考虑这句话是否可以被验证或证明。如果这句话涉及虚构的仙侠元素，例如魔法、修炼法术等，无法通过现实的观察、实验或证据来验证，那么它更有可能属于虚拟仙侠世界。如果这句话涉及现实生活中的事实、事件或已知的真实信息，可以通过查证或验证来确认，那么它更有可能属于真实世界。
>
>语言风格和表达方式：注意这句话所使用的语言风格和表达方式。虚拟仙侠世界通常有其独特的修辞、词汇和叙事风格，可能包含与现实世界不同的词语、短语或俗语。如果这句话使用了仙侠世界中常见的特定词汇或短语，那么它更有可能属于虚拟仙侠世界。
>
>请注意，以上是一些常见的判断指标，但并非绝对可靠。有些情况下，一句话可能具有模糊或混合的特征，使其不容易确定它所属的世界。在判断时需要综合考虑上述因素，并根据具体情境和语境进行判断。

采取分而治之的方法:
 1. 提取句子中的<人名>，<物品>，以及<事件>
 2. 使用搜索API获取对应<人名>，<物品>，以及<事件>的世界观背景
 3. 排序搜索结果
 4. 归纳得出最终结论
 
![main idea](https://raw.githubusercontent.com/openai/openai-cookbook/de3bd5843433a8240f9bcb8e275ccb9dac7f7f0a/images/search_rerank_answer.png)

https://github.com/openai/openai-cookbook/blob/main/examples/Question_answering_using_a_search_API.ipynb


In [1]:
# installations
!pip install openai google-api-python-client

You should consider upgrading via the 'pip install --upgrade pip' command.


# Prerequiste
## openai key
add an api from openai console: https://platform.openai.com/account/api-keys
## google search API
add customsearch api: https://console.cloud.google.com/apis/library/customsearch.googleapis.com

In [2]:
%env OPENAI_API_KEY="<openai_api_key>"
%env GOOGLE_SEARCH_API_KEY="<openai_api_key>"

env: OPENAI_API_KEY="<openai_api_key>"
env: GOOGLE_SEARCH_API_KEY="<openai_api_key>"


## 1. 提取句子中的<人名>，<物品>，以及<事件>

In [4]:
import os
import openai
from functools import lru_cache


openai_api_key = os.getenv("OPENAI_API_KEY")
GPT_MODEL = "gpt-3.5-turbo"

# 读取json形式的chatGPT回答
@lru_cache(maxsize=None)
def json_gpt(input: str, verbose: bool = True):
    completion = openai.ChatCompletion.create(
        model=GPT_MODEL,
        messages=[
            {"role": "system", "content": "You are ChatGPT, a large language model trained by OpenAI. Answer as concisely as possible. Knowledge cutoff: {knowledge_cutoff} Current date: {current_date}"},
            {"role": "user", "content": input},
        ],
        stop='\n'
    )

    text = completion.choices[0].message.content
    if verbose:
        print(text)
    parsed = json.loads(text)

    return parsed

In [5]:
USER_QUESTION = "如何学会豪火球术"

In [6]:
# 提取句子中的<人名>，<物品>，以及<事件>
import json


QUERIES_INPUT = f"""
要求：提取以下语句中人物、物品以及事件
输出：仅输出json格式，无须解释
语句:  {USER_QUESTION}

Format: {{"人物": ["人物1", "人物2"], "物品":["物品1"], "事件": ["事件1", "事件2"]}}
"""

queries = json_gpt(QUERIES_INPUT)

queries

{"人物": [], "物品": ["豪火球术"], "事件": ["学会"]}


{'人物': [], '物品': ['豪火球术'], '事件': ['学会']}

# 2. 用google search API查找相关关键词

In [7]:
# 用谷歌查找对应<人物>、<物品>以及<事件>
from googleapiclient.discovery import build
import itertools


def search_google(
    query: str,
    developerKey: str = os.getenv("GOOGLE_SEARCH_API_KEY"),
    cx: str = "f68edb5b01f854626",  # a search engine key
) -> dict:
    service = build(
        "customsearch", "v1", developerKey=developerKey,
    )

    res = (
        service.cse()
        .list(
            q=query,
            cx=cx,
        )
        .execute()
    )
    
    results = []
    for item in res["items"]:
        results.append({"title": item["title"], "description": item["snippet"], "url": item["link"]})
    return results


items = []
flat_queries = list(itertools.chain.from_iterable(queries.values()))
for query in flat_queries:
    if query:
        items.extend(search_google(query))
        
items

[{'title': '火遁·豪火球之术_百度百科',
  'description': '豪火球之术，日本漫画《火影忍者》及其衍生作品中宇智波一族常用的火遁忍术，等级为C（中忍级别），《临之书》称之为高难度忍术。查克拉聚集在喉咙后，从口中向前方\xa0...',
  'url': 'https://baike.baidu.com/item/%E7%81%AB%E9%81%81%C2%B7%E8%B1%AA%E7%81%AB%E7%90%83%E4%B9%8B%E6%9C%AF/18264758'},
 {'title': '豪火球术，这个术是《火影》里最常用的火遁',
  'description': 'Jun 19, 2018 ... 这是第一种破解豪火球术的方式——闪避，也是最常见的一种，毕竟豪火球术只是个直线大范围攻击，只要判断准了攻击方向和路径，闪避是很容易做到的。',
  'url': 'http://www.sohu.com/a/236556548_100170596'},
 {'title': '豪火球术_百度百科',
  'description': '豪火球术是忍法中的火遁。',
  'url': 'https://baike.baidu.com/item/%E8%B1%AA%E7%81%AB%E7%90%83%E6%9C%AF/1076038'},
 {'title': '火遁豪火球之术怎么发音的，顺便讲下印',
  'description': '豪火球术，豪火球の术--ごうがきゅうのじゅつ，豪火球之术结印式：巳－未－申－亥－午－寅。 豪火球之术，日本漫画《火影忍者》及其衍生作品中宇智波一族常用的火遁忍\xa0...',
  'url': 'https://zhidao.baidu.com/question/27386150'},
 {'title': '火影忍术结印赶紧学起来- 知乎',
  'description': 'Nov 14, 2020 ... 首先给大家介绍几个最基本的手印。 一·火遁豪火球之术：巳－未－申－亥－午－寅。从口中吐出巨大火球攻击敌人的忍术，其威力可以烧出直径7公尺的洞来\xa0...',
  'url': 'https://zhuanlan.zhihu.com/p/293394837'},
 {'title': 

# 3. 相关度排序
相关度 = cosine_similarity(sentence_embedding(<虚拟回答>), sentence_embedding(<搜索文本>))

In [8]:
HA_INPUT = f"""
Generate a hypothetical answer to the user's question. This answer will be used to rank search results. 
Don't give final conclusion but some steps instead. Don't use any actual facts. Use placeholders like 人物, 物品, 事件. Pretend you have search results of the above placeholders. 

User question: 如何判断以下这句话属于现实世界还是虚拟仙侠世界: {USER_QUESTION}

Format: {{"hypotheticalAnswer": "虚拟回答"}}
"""

hypothetical_answer = json_gpt(HA_INPUT)["hypotheticalAnswer"]

hypothetical_answer

{"hypotheticalAnswer": "通过以下步骤可以判断以下这句话属于现实世界还是虚拟仙侠世界：\n\n1. 查阅人物资料：了解是否存在能够施展豪火球术的人物\n\n2. 研究物品信息：探究是否存在相关的法术书籍、魔法器具等\n\n3. 探索事件历史：寻找是否有有关豪火球术的历史事件记录\n\n以上步骤可以为判断提供一些线索，但最终结论需要综合多方面的证据。"}


'通过以下步骤可以判断以下这句话属于现实世界还是虚拟仙侠世界：\n\n1. 查阅人物资料：了解是否存在能够施展豪火球术的人物\n\n2. 研究物品信息：探究是否存在相关的法术书籍、魔法器具等\n\n3. 探索事件历史：寻找是否有有关豪火球术的历史事件记录\n\n以上步骤可以为判断提供一些线索，但最终结论需要综合多方面的证据。'

In [9]:
from typing import List


def embeddings(input: List[str]) -> List[List[str]]:
    response = openai.Embedding.create(model="text-embedding-ada-002", input=input)
    return [data.embedding for data in response.data]


hypothetical_answer_embedding = embeddings(hypothetical_answer)[0]
item_embeddings = embeddings(
    [
        f"{item['title']} {item['description']}"
        for item in items
    ]
)

In [10]:
# Calculate cosine similarity
import numpy as np


cosine_similarities = []
for item_embedding in item_embeddings:
    cosine_similarities.append(np.dot(hypothetical_answer_embedding, item_embedding))

cosine_similarities[0:10]

[0.8180832919433096,
 0.828616366337815,
 0.8096793495181102,
 0.80669858217406,
 0.8268265795803552,
 0.8285931977432259,
 0.8280657819001908,
 0.8350377616547668,
 0.8208030354381239,
 0.8180600158683436]

## 只选择相关度最高的top5的结果，降低chatgpt文本数量

In [11]:
cored_items = zip(items, cosine_similarities)

# Sort articles by cosine similarity
sorted_items = sorted(cored_items, key=lambda x: x[1], reverse=True)

# Print top 5 items in similarity
print("Top 5 items:", "\n")

for article, score in sorted_items[0:5]:
    print("Title:", article["title"])
    print("Description:", article["description"])
    print("Score:", score)
    print()

Top 5 items: 

Title: 豪火球之术真人版_火遁豪火球之术真人版_卡通豪火球之术中文-抖音
Description: Dec 18, 2022 ... 您在查找“豪火球之术真人版”吗？抖音短视频，帮你找到更多精彩视频内容！让每一个人看见并连接更大的世界，让现实生活更美好。
Score: 0.8350377616547668

Title: 豪火球术，这个术是《火影》里最常用的火遁
Description: Jun 19, 2018 ... 这是第一种破解豪火球术的方式——闪避，也是最常见的一种，毕竟豪火球术只是个直线大范围攻击，只要判断准了攻击方向和路径，闪避是很容易做到的。
Score: 0.828616366337815

Title: 豪火球术的结印顺序
Description: 豪火球术的结印顺序 ...展开 ...收起. 配图加50分 ...收起. 提问. 答题 有奖励. 共1个回答. 提交优质回答，最高可获得现金3元. 查看规则. 收起有奖发布. 问题不好答？
Score: 0.8285931977432259

Title: 火影：动漫里豪火球术要从嘴里喷火，卡卡西用此术，为何无损面罩 ...
Description: Sep 4, 2020 ... 旗木卡卡西使用火遁·豪火球之术时，面罩没有被火烧毁，就是这部动漫里众多未解之谜之一。动漫里设定了三大瞳术，宇智波家族的写..._新浪网.
Score: 0.8280657819001908

Title: 火影忍术结印赶紧学起来- 知乎
Description: Nov 14, 2020 ... 首先给大家介绍几个最基本的手印。 一·火遁豪火球之术：巳－未－申－亥－午－寅。从口中吐出巨大火球攻击敌人的忍术，其威力可以烧出直径7公尺的洞来 ...
Score: 0.8268265795803552



# 4. 根据搜索结果生成最后的回答

In [12]:
formatted_top_results = [
    {
        "title": item["title"],
        "description": item["description"],
        "url": item["url"],
    }
    for item, _ in sorted_items[0:5]
]

In [13]:
from IPython import display  # for pretty printing


ANSWER_INPUT = f"""
Generate an answer to the user's question based on the given search results. 
TOP_RESULTS: {formatted_top_results}
USER_QUESTION: 如何判断以下这句话属于现实世界还是虚拟仙侠世界: {USER_QUESTION}

Include as much information as possible in the answer. Reference the relevant search result urls as markdown links. Provide probabilities if it is a classification problem.

Format: {{"属于虚拟世界的概率(%)": "probability", "detailed": "详细解释"}}
"""

completion = openai.ChatCompletion.create(
    model=GPT_MODEL,
    messages=[
        {"role": "system", "content": "You are ChatGPT, a large language model trained by OpenAI. Answer as concisely as possible. Knowledge cutoff: {knowledge_cutoff} Current date: {current_date}"},
        {"role": "user", "content": ANSWER_INPUT},
    ],
    temperature=0.5,
    stream=True,
)

text = ""
for chunk in completion:
    text += chunk.choices[0].delta.get("content", "")
    display.clear_output(wait=True)
    display.display(display.Markdown(text))


{"属于虚拟世界的概率(%)": "80%", "detailed": "根据搜索结果，豪火球术是火影动漫中的忍术技能，属于虚拟世界。相关搜索结果包括豪火球术的破解方式、结印顺序以及在动漫中的使用情况。参考链接：[豪火球术，这个术是《火影》里最常用的火遁](http://www.sohu.com/a/236556548_100170596)，[火影：动漫里豪火球术要从嘴里喷火，卡卡西用此术，为何无损面罩](https://k.sina.com.cn/article_7176811671_1abc5749700100puqv.html?from=animation)，[火影忍术结印赶紧学起来- 知乎](https://zhuanlan.zhihu.com/p/293394837)"}